# problem: To compare major indian cities 
### For people thinking of opening their business in major cities(most populated cities) or people planning to transfer from one location to another


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # for bar graph and pie charts i.e visual analysis
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#foursqare credentials
CLIENT_ID = 'FCBYPCDQD3QRIVYNVSWTMSNLJMARGVMBUH1QESH0MSIPYASC' # Foursquare ID
CLIENT_SECRET = 'PHXQL0ENI5ZM3B1QAJSA2G54DLCMPBBCYXOUFA240CUPTOBG' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FCBYPCDQD3QRIVYNVSWTMSNLJMARGVMBUH1QESH0MSIPYASC
CLIENT_SECRET:PHXQL0ENI5ZM3B1QAJSA2G54DLCMPBBCYXOUFA240CUPTOBG


In [4]:
df1=pd.read_csv('in.csv')
df1.set_index('city',inplace=True)
df1['population'].head(30).plot(kind='bar',color='lightskyblue',figsize=(15,6))
plt.title('Most Populated cities',size=30)
plt.xlabel('City',size=20)
plt.ylabel('Population',x=1.12,size=20)

FileNotFoundError: [Errno 2] File b'in.csv' does not exist: b'in.csv'

In [ ]:
df=pd.read_csv('in.csv')
df=df.head(30)
df=df[['city','lat','lng']]
df

In [ ]:
lat=20.5937
long= 78.9629

In [ ]:
India = folium.Map(location=[lat,long],zoom_start=5)
for lat, lng, label in zip(df['lat'], df['lng'], df['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(India)  
    
India

In [ ]:
LIMIT=100

In [ ]:
def getvenues(names, latitudes, longitudes, radius=2000):
    venues_list=[]
    for lat, lng, label in zip(latitudes,longitudes,names):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            label, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
total_venues=getvenues(names=df['city'],
                       latitudes=df['lat'],
                       longitudes=df['lng'])


In [ ]:
total_venues.head()

In [ ]:
total_onehot = pd.get_dummies(total_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
total_onehot['Neighborhood'] = total_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [total_onehot.columns[96]] + list(total_onehot.columns[1:95])+list(total_onehot.columns[97:])
total_onehot = total_onehot[fixed_columns]

total_onehot.head()

In [ ]:
total_grouped = total_onehot.groupby('Neighborhood').mean().reset_index()
total_grouped

In [ ]:
total_grouped['Art']=total_grouped['Art Gallery']+total_grouped['Art Museum']+total_grouped['Historic Site']+total_grouped['History Museum']
total_grouped.drop(['Art Gallery','Art Museum','Historic Site','History Museum'],axis=1,inplace=True)

In [ ]:
total_grouped['Western food']=total_grouped['American Restaurant']+total_grouped['BBQ Joint']+total_grouped['Burger Joint']+total_grouped['Donut Shop']+total_grouped['French Restaurant']+total_grouped['Fried Chicken Joint']+total_grouped['Mexican Restaurant']+total_grouped['Pizza Place']+total_grouped['Steakhouse']
total_grouped.drop(['American Restaurant','BBQ Joint','Burger Joint','Donut Shop','French Restaurant','Fried Chicken Joint','Mexican Restaurant','Pizza Place','Steakhouse'],axis=1,inplace=True)

In [ ]:
total_grouped['Eastern food']=total_grouped['Afghan Restaurant']+total_grouped['Asian Restaurant']+total_grouped['Chaat Place']+total_grouped['Chinese Restaurant']+total_grouped['Diner']+total_grouped['Food']+total_grouped['Food Court']+total_grouped['Hotel']+total_grouped['Indian Restaurant']+total_grouped['Indian Sweet Shop']+total_grouped['Japanese Restaurant']+total_grouped['Punjabi Restaurant']+total_grouped['Sandwich Place']+total_grouped['Sushi Restaurant']+total_grouped['Thai Restaurant']
total_grouped.drop(['Afghan Restaurant','Asian Restaurant','Chaat Place','Chinese Restaurant','Diner','Food','Food Court','Hotel','Indian Restaurant','Indian Sweet Shop','Japanese Restaurant','Punjabi Restaurant','Sandwich Place','Sushi Restaurant','Thai Restaurant'],axis=1,inplace=True)

In [ ]:
total_grouped.head()

In [ ]:
kclusters = 6
#k=6 seems appropriate( for 20 cities) grouping of cities change on the basis of the number of cities you take in the start
total_grouped_clustering = total_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
#total_grouped.drop(['Cluster Labels'],axis=1,inplace=True) #uncomment that command if excecuting again after changing K while taking more number of cities
total_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

total_grouped.head()

In [ ]:
total_venues=total_venues[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']]
total_merged=total_grouped
total_merged = total_merged.join(total_venues.set_index('Neighborhood'), on='Neighborhood')


In [ ]:
map_clusters = folium.Map(location=[lat, long], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(total_merged['Neighborhood Latitude'], total_merged['Neighborhood Longitude'], total_merged['Neighborhood'], total_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analysis 

In [ ]:
total_new=total_merged.groupby(['Neighborhood']).sum()
total_new['Western food']=sorted(total_new['Western food'],reverse=True)

In [ ]:
colors_list = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen', 'pink','violet','cyan','deepskyblue','yellow']
explode_list = [0.2, 0.2, 0.1, 0.1, 0.1,0,0,0,0,0]
legend=total_new.index[0:10]
total_new['Western food'].head(10).plot(kind='pie',
                            figsize=(15, 6),
                            labels=None,
                            autopct='%1.1f%%', # add in percentages
                            startangle=90,     # start angle 90° 
                            shadow=True,
                            colors=colors_list,
                            explode=explode_list)

plt.title('Western food in different cities')
plt.axis('equal') # Sets the pie chart to look like a circle.
plt.legend(labels=legend,loc='upper left')
plt.ylabel('')

plt.show()